# Huc 12 Predictors
## Test 19020302 HUC 8 incl chugach nforest
All Huc 12s- Intersect w/catchments and find catchment (find downstream most catchment) with most upstream contribution and build watersheds for all ds catchments

Keep -
mnwtd_slope
% lake
summer precip - Runs in R but need downstream point (centroid of ds catchment)
predict for huc8

Visualize by HUC12 - Join data back to Huc12

## Section
Import modules
Set/Create Workspaces
Collect Data
Merge Data



## Section 2
### By Vector Processing Unit
Identify downstream-most catchment for each Huc 12
 * Select by location and select catchment with most us contributing area
    * May be stored in VAA for NHDPlus
    * Stored in Stream
 * Generate Centroid point and append to centroid dataset
    * Retain cat_id and Huc12-id
Append to HC12 catchment dataset



## Section 3
### Generate watersheds
Iterate over
